# Data Collection and Preparation

# Mary Donovan Martello

## The first part of this project consisted of collecting, cleaning and preparing data from three different sources: a flat file, a website (collected by scraping the website), and an API. The final product included joining all three sources on the same key, storing them in a SQLite database and executing SQL commands. This notebook includes scraping a website for data and cleaning the data.

# Part 2:  Website Data Source / Cleaning

In [1]:

%matplotlib inline

# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup


## The data for this notebook is mutual fund financial performance located at: https://charts.ussif.org/mfpc/

### Use Beautiful Soup to scrape the website.

In [2]:
#Read the page using bs4

with open("Mutual Fund Performance Chart _ US SIF.html", "r", encoding = 'utf-8') as fd:
    soup = BeautifulSoup(fd)    
    #print(soup.prettify())

In [3]:
# Find the table structure using bs4

allTables = soup.find_all('table')
print("Total number of <table> --- {}".format(len(allTables)))

Total number of <table> --- 10


In [4]:
# Find the right table using bs4
myTable = soup.find("table", attrs={"id" : "FinancialPerformance"})

## Separate the Header Titles and the Corresponding Data

**Find the first table row in the found table.**

In [5]:
#  1. find the first table row in the found table
titles = myTable.tbody.findAll('tr', recursive=False)[0]

**Get a list of header cells; the header cells have the header titles in them.**

In [6]:
#  2. get a list of header cells (th) in the first table row found; the header cells have the header titles in them
header = [th.getText().strip() for th in titles.findAll('th')]
header

['Ticker',
 'Fund Name',
 'Fund Type',
 'InceptionMonth',
 'AUM(US$ millions)',
 'YTD %',
 '1 yrAvg %',
 '3 yrAvg %',
 '5 yrAvg %',
 '10 yrAvg %',
 'Prev YrRtn %',
 'MgmtFee %',
 'ExpRatio %',
 'StdDev',
 'Benchmark Used']

**Get all the data.**

In [7]:
# 3. Get all the data
data = myTable.findAll('tbody', recursive=False)

**Put the data into a list.**

In [8]:
# 4. Put the data into a list
dataList = []
for tr in data:
    dataList.append(myTable.tbody.findAll('tr'))

**Get the data cells from the data list.**

In [9]:
# 5. Get the data cells from the data list; these have the data that make up the chart.
dataCells = dataList[0][1].findAll('td')[0:]

**Extract the text from the data cells.**

In [10]:
# 6. extract the text from the data cells
dataCellsText = [td.get_text().strip() for td in tr.findAll('td')]

### Convert the data from a list to an array.

In [11]:
# Convert the data from a list to an array; data from website is in the shape of (1, 2640) instead of (176, 15)
rowsNo = int(len(dataCellsText)/15)
colsNo = int(len(header))
dataArray = np.array(dataCellsText)
shape = (rowsNo, colsNo)
finalData = dataArray.reshape(shape)

### Convert the array to a Pandas Dataframe.

In [12]:
webDf = pd.DataFrame(finalData, columns=header)
webDf.head(20)

,Ticker,Fund Name,Fund Type,InceptionMonth,AUM(US$ millions),YTD %,1 yrAvg %,3 yrAvg %,5 yrAvg %,10 yrAvg %,Prev YrRtn %,MgmtFee %,ExpRatio %,StdDev,Benchmark Used
0,SSIAX,1919 Socially Responsive Balanced Fund - A,Balanced,11/1992,$237.41M,-3.05,5.61,9.52,7.22,7.94,24.69,0.64,1.25,14.97,Blend: 60% S&P 500 - 40% BC Aggregate
1,SESLX,1919 Socially Responsive Balanced Fund - C,Balanced,5/1993,$237.41M,-3.26,4.89,8.75,6.45,7.2,23.78,0.64,1.97,14.98,Blend: 60% S&P 500 - 40% BC Aggregate
2,LMRNX,1919 Socially Responsive Balanced Fund - I,Balanced,7/2008,$237.41M,-2.94,6,9.88,7.53,8.23,25.1,0.64,0.96,14.99,Blend: 60% S&P 500 - 40% BC Aggregate
3,ACASX,Access Capital Community Investment Fund A,Bond(Fixed Inc),1/2009,$544.77M,2.87,6.56,2.82,2.13,2.49,5.63,0.35,1.11,1.58,Barclays Capital US Securitized Bond
4,ACCSX,Access Capital Community Investment Fund I,Bond(Fixed Inc),7/1998,$544.77M,2.87,6.82,3.13,2.37,2.64,6.12,0.35,0.66,1.57,Barclays Capital US Securitized Bond
5,APPLX,Appleseed Fund,Int'lGlobal,12/2006,$104.34M,-25.04,-16.98,-2.64,-1.61,2.51,15.68,0.85,1.2,32.54,MSCI World
6,APPIX,Appleseed Fund Institutional,Int'lGlobal,2/2011,$104.34M,-25,-16.87,-2.48,-1.43,--,15.85,0.85,1.06,32.62,MSCI World
7,ASUIX,Arabesque Systematic USA Fund,Balanced,5/2017,$27.75M,-12.64,-11.16,3.41,--,--,15.32,0.75,0.95,14.78,MSCI USA
8,CAAPX,Ariel Appreciation Fund,EquityMid-Sm Cap,12/1989,$953.56M,-22.44,-20.44,-3.63,-0.67,6.53,24.6,0.69,1.14,32.62,Russell Mid Cap Value
9,ARFFX,Ariel Focus Fund,AllCap,6/2005,$44.61M,-21.88,-17.23,-2.4,0.34,5.42,25.62,0.65,1,34.69,Russell 1000 Value


In [13]:
webDf.shape

(176, 15)

### Explore the data.

In [14]:
count = -1
for c in webDf.columns:
    count +=1
    print(type(webDf.iloc[5, count]))
    

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [15]:
for c in webDf.columns:
    miss = webDf[c].isnull().sum()
    if miss>0:
        print("{} has {} missing value(s)".format(c,miss))

## Clean the Data

### Replace Headers.

### Rename Column Names.

In [16]:
webDf.columns

Index(['Ticker', 'Fund Name', 'Fund Type', 'InceptionMonth',
       'AUM(US$ millions)', 'YTD %', '1 yrAvg %', '3 yrAvg %', '5 yrAvg %',
       '10 yrAvg %', 'Prev YrRtn %', 'MgmtFee %', 'ExpRatio %', 'StdDev',
       'Benchmark Used'],
      dtype='object')

In [17]:
webDf.rename({'Ticker': 'fund_name', 'Fund Name': 'fund_extended_name', 'AUM(US$ millions)': 'net_assets', 
               'InceptionMonth': 'inception_date', 'YTD %': 'fund_return_ytd', '1 yrAvg %': 'fund_return_1year', 
               '3 yrAvg %': 'fund_return_3years', '5 yrAvg %': 'fund_return_5years', '10 yrAvg %': 'fund_return_10years', 
               'ExpRatio %': 'net_annual_expense_ratio_fund'}, axis=1, inplace=True)

In [18]:
webDf.columns

Index(['fund_name', 'fund_extended_name', 'Fund Type', 'inception_date',
       'net_assets', 'fund_return_ytd', 'fund_return_1year',
       'fund_return_3years', 'fund_return_5years', 'fund_return_10years',
       'Prev YrRtn %', 'MgmtFee %', 'net_annual_expense_ratio_fund', 'StdDev',
       'Benchmark Used'],
      dtype='object')

### Strip Strings

In [19]:
# Remove $ and M from value in 'net_assets' series
webDf['net_assets']

0      $237.41M
1      $237.41M
2      $237.41M
3      $544.77M
4      $544.77M
         ...   
171    $213.13M
172     $60.31M
173          --
174     $49.55M
175     $10.12M
Name: net_assets, Length: 176, dtype: object

In [20]:
# Remove $ and M from value in 'net_assets' series cont.
webDf['net_assets'] = webDf['net_assets'].map(lambda x: x[1:-1]).str.strip()

In [21]:
webDf['net_assets']

0      237.41
1      237.41
2      237.41
3      544.77
4      544.77
        ...  
171    213.13
172     60.31
173          
174     49.55
175     10.12
Name: net_assets, Length: 176, dtype: object

### Replace symbols with integers

In [22]:
# Replace '--' with 0 in all rows and columns
webDf = webDf.replace('--', 0)

In [23]:
webDf.iloc[175, 6]

0

### Convert strings to floats

In [24]:
# convert data types from strings to floats
webDf.iloc[:, 5] = webDf.iloc[:, 5].astype(float)
webDf.iloc[:, 6] = webDf.iloc[:, 6].astype(float)
webDf.iloc[:, 7] = webDf.iloc[:, 7].astype(float)
webDf.iloc[:, 8] = webDf.iloc[:, 8].astype(float)
webDf.iloc[:, 9] = webDf.iloc[:, 9].astype(float)
webDf.iloc[:, 10] = webDf.iloc[:, 10].astype(float)
webDf.iloc[:, 11] = webDf.iloc[:, 11].astype(float)
webDf.iloc[:, 12] = webDf.iloc[:, 12].astype(float)
webDf.iloc[:, 13] = webDf.iloc[:, 13].astype(float)

In [26]:
# loop to see what value is causing the ValueError
    # converting the series in the [:, 4] location resulted in an error of: ValueError: could not convert string to float
    # convert the Series to a list to be able to use Enumerate
assetsList = webDf.iloc[:, 4].tolist()
for index, val in enumerate(assetsList):
    try:
        float(val)
    except ValueError:
        print('Line {i} is the problem.'.format(i = index))
        break

Line 173 is the problem.


In [26]:
# fix webDf.iloc[173, 4]
webDf.iloc[173, 4]

''

In [27]:
# fix webDf.iloc[173, 4] cont.
webDf.iloc[173, 4] = webDf.iloc[173, 4].replace('', '0')

In [28]:
# retry converting webDf.iloc[:, 4] = webDf.iloc[:, 4].astype(float)
webDf.iloc[:, 4] = webDf.iloc[:, 4].astype(float)

### Convert date format in inception_date to year only

In [29]:
# convert date format in inception_date to year only
webDf['inception_date']

0      11/1992
1       5/1993
2       7/2008
3       1/2009
4       7/1998
        ...   
171     6/1999
172     8/2011
173    10/2008
174     6/2012
175     5/2019
Name: inception_date, Length: 176, dtype: object

In [30]:
# convert date format of inception_date to year only cont.
import datetime

webDf['inception_date'] = pd.to_datetime(webDf['inception_date'])

webDf['inception_date'] = webDf['inception_date'].dt.year

In [31]:
webDf['inception_date']

0      1992
1      1993
2      2008
3      2009
4      1998
       ... 
171    1999
172    2011
173    2008
174    2012
175    2019
Name: inception_date, Length: 176, dtype: int64

### Convert net_assets values to millions.

In [32]:
# convert net_assets values to millions
webDf['net_assets']

0      237.41
1      237.41
2      237.41
3      544.77
4      544.77
        ...  
171    213.13
172     60.31
173      0.00
174     49.55
175     10.12
Name: net_assets, Length: 176, dtype: float64

In [33]:
# convert net_assets values to millions cont.
webDf['net_assets'] = webDf['net_assets'] * 1000000

In [34]:
webDf['net_assets']

0      237410000.0
1      237410000.0
2      237410000.0
3      544770000.0
4      544770000.0
          ...     
171    213130000.0
172     60310000.0
173            0.0
174     49550000.0
175     10120000.0
Name: net_assets, Length: 176, dtype: float64

In [35]:
webDf.head(5)

,fund_name,fund_extended_name,Fund Type,inception_date,net_assets,fund_return_ytd,fund_return_1year,fund_return_3years,fund_return_5years,fund_return_10years,Prev YrRtn %,MgmtFee %,net_annual_expense_ratio_fund,StdDev,Benchmark Used
0,SSIAX,1919 Socially Responsive Balanced Fund - A,Balanced,1992,237410000.0,-3.05,5.61,9.52,7.22,7.94,24.69,0.64,1.25,14.97,Blend: 60% S&P 500 - 40% BC Aggregate
1,SESLX,1919 Socially Responsive Balanced Fund - C,Balanced,1993,237410000.0,-3.26,4.89,8.75,6.45,7.20,23.78,0.64,1.97,14.98,Blend: 60% S&P 500 - 40% BC Aggregate
2,LMRNX,1919 Socially Responsive Balanced Fund - I,Balanced,2008,237410000.0,-2.94,6.00,9.88,7.53,8.23,25.10,0.64,0.96,14.99,Blend: 60% S&P 500 - 40% BC Aggregate
3,ACASX,Access Capital Community Investment Fund A,Bond(Fixed Inc),2009,544770000.0,2.87,6.56,2.82,2.13,2.49,5.63,0.35,1.11,1.58,Barclays Capital US Securitized Bond
4,ACCSX,Access Capital Community Investment Fund I,Bond(Fixed Inc),1998,544770000.0,2.87,6.82,3.13,2.37,2.64,6.12,0.35,0.66,1.57,Barclays Capital US Securitized Bond


In [36]:
webDf.shape

(176, 15)

In [37]:
# export datafram to a csv file
webDf.to_csv (r'C:\dev\code\540FinalProject\web_dataframe.csv', index = False, header=True)